In [7]:
import pandas as pd 
import numpy as np
import string, re
import nltk
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from time import time

%matplotlib inline

In [45]:
data = pd.read_csv('data/first-gop-debate-twitter-sentiment/sentiment.csv')
emoji_list = pd.read_csv('data/emoji_table.txt', encoding='utf-8', index_col=0).index.values
data.head(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [44]:
# List of positive and negative tweets
sad = [':‑(', ':(', ':‑c', ':c', ':‑<', ':<', ':‑[' ,':[', ':-||', '>:[', ':{', ':@', '>:(']
Positive = [':‑)',':)', ':-]', ':]',':-3', ':3', ':->', ':>' ,'8-)', '8)',':-}', ':}', ':o)', ':c)', ':^)' ,'=]', '=)'
           ,':‑D', ':D', '8‑D', '8D', 'x‑D', 'xD', 'X‑D', 'XD', '=D', '=3', 'B^D']

In [46]:
stop_list = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
# rt - stands for retweet
stop_list = stop_list +["rt", 'url']

# regex for capturing tweets
# reg = '(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)'
emoticons = "|".join(map(re.escape, sad + Positive))

emoji_pattern = re.compile(u'('
    u'\ud83c[\udf00-\udfff]|'
    u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
    u'[\u2600-\u26FF\u2700-\u27BF])+', 
    re.UNICODE)
classifier =[]
def preprocess(tweet):
    # only processing if the the value is a string
    if type(tweet)!=type(2.0):
        tweet = tweet.decode('utf-8').strip()
        tweet = tweet.lower()
        # Removing hashtags
        tweet = " ".join(tweet.split('#'))
        # Removing URLs
        tweet = re.sub('((www\.[^\s]+)|(https://[^\s]+))','',tweet)
        tweet = re.sub("(http\S+)|(https\S+)", '', tweet)
        # Removing User mentions
        tweet = re.sub('@[^\s]+','',tweet)
        tweet = tweet.strip('\'"')
        # Removing stop words - This can be moved to count vectorization
        tweet  = " ".join([word for word in tweet.split(" ") if word not in stop_list])
        # lemmatizing words 
        tweet = " ".join([lemmatizer.lemmatize(word) for word in tweet.split(" ")])
    else:
        tweet=''
    return tweet

def extractEmoticons(tweet):
    # emoji = emoji_pattern.findall(tweet)
    emoji = []
    for emo in emoji_list:
        if emo in tweet:
            emoji.append(emo)
    
    # these are :) :-) and other stuff
    emoticons = re.findall(reg, tweet)
    return " , ".join(emoji + emoticons)
def removeEmoticons(tweet):
    return re.sub(reg,'',tweet)

# data = data.dropna()
data['processed_text'] = data.text.apply(preprocess)

#getting the emoticons from the cleaned data
data['emoticons'] = data['processed_text'].apply(extractEmoticons)

# Removing emoticons from the text data
data['processed_text'] = data['processed_text'].apply(removeEmoticons)

In [8]:
vectorizer = TfidfVectorizer(stop_words=stop_list)
X = vectorizer.fit_transform(data.processed_text)

In [9]:
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1,
                verbose=True)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(3):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, -10:]:
        print(' %s' % terms[ind])
    print()

Clustering sparse data with KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=3, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=True)
Initialization complete
Iteration  0, inertia 26488.945
Iteration  1, inertia 13590.235
Iteration  2, inertia 13573.981
Iteration  3, inertia 13558.029
Iteration  4, inertia 13547.756
Iteration  5, inertia 13539.660
Iteration  6, inertia 13538.935
Iteration  7, inertia 13538.329
Iteration  8, inertia 13538.170
Iteration  9, inertia 13538.093
Iteration 10, inertia 13538.071
Iteration 11, inertia 13538.060
Iteration 12, inertia 13538.049
Iteration 13, inertia 13538.031
Iteration 14, inertia 13538.024
Iteration 15, inertia 13538.015
Iteration 16, inertia 13537.991
Iteration 17, inertia 13537.987
Iteration 18, inertia 13537.980
Iteration 19, inertia 13537.969
Iteration 20, inertia 13537.963
Iteration 21, inertia 13537.953
Iteration 22, inertia 13537.946
Iteration 23, inertia 13537.920
Iteration 24

In [47]:
data[data['emoticons'] != '']

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone,processed_text,emoticons
61,62,No candidate mentioned,1.0000,yes,1.0000,Negative,1.0000,FOX News or Moderators,1.0000,NaN,...,NaN,NaN,RT @AmyMek: Status 👉 Single \n\nI broke up wi...,NaN,2015-08-07 09:54:23 -0700,629697105838383104,"Florida,, usually",Eastern Time (US & Canada),status 👉 single \n\ni broke last night! g...,👉
125,126,Donald Trump,1.0000,yes,1.0000,Negative,0.6549,FOX News or Moderators,1.0000,NaN,...,NaN,NaN,RT @AmyMek: We all owe @realDonaldTrump a huge...,NaN,2015-08-07 09:53:53 -0700,629696979858292736,NaN,Pacific Time (US & Canada),owe huge thank 4 exposing world👉 chris wall...,👉
140,141,Donald Trump,0.3921,yes,0.6262,Negative,0.6262,None of the above,0.3921,NaN,...,NaN,NaN,"RT @VH1: Honestly, Donald Trump you were ALL t...",NaN,2015-08-07 09:53:47 -0700,629696952985350144,NaN,Cape Verde Is.,"honestly, donald trump type petty last night'...",😒
143,144,No candidate mentioned,0.4510,yes,0.6716,Positive,0.3372,FOX News or Moderators,0.2264,NaN,...,NaN,NaN,RT @peddoc63: Go Carly📢\nGo Carly📢\nGo Carly...,NaN,2015-08-07 09:53:46 -0700,629696947918643200,NaN,NaN,go carly📢\ngo carly📢\ngo carly📢\n carly20...,📢
148,149,Donald Trump,1.0000,yes,1.0000,Negative,0.3385,None of the above,1.0000,NaN,...,NaN,NaN,RT @DrMartyFox: #Trump Wins Drudge \nDebate Po...,NaN,2015-08-07 09:53:44 -0700,629696941769625600,SSCA,Arizona,trump win drudge \ndebate poll \n\n➡️ landsl...,"➡️ , ☑️ , 🇺🇸"
160,161,Donald Trump,0.4347,yes,0.6593,Neutral,0.3626,None of the above,0.4347,NaN,...,NaN,NaN,RT @STEEL5757: 🎀 #DonaldTrump Plaza Casino #T...,NaN,2015-08-07 09:53:39 -0700,629696921444028417,NaN,NaN,🎀 donaldtrump plaza casino teddybear plush...,🎀
184,185,No candidate mentioned,1.0000,yes,1.0000,Negative,1.0000,FOX News or Moderators,0.6629,NaN,...,NaN,NaN,RT @renomarky: ☑ never watch #KellyFile again\...,NaN,2015-08-07 09:53:26 -0700,629696865982902272,NaN,Central Time (US & Canada),☑ never watch kellyfile again\n\n1. gopdeba...,🇺🇸
201,202,No candidate mentioned,1.0000,yes,1.0000,Negative,1.0000,FOX News or Moderators,1.0000,NaN,...,NaN,NaN,"RT @realDonaldTrump: ""@FrankLuntz: I'm getting...",NaN,2015-08-07 09:53:18 -0700,629696830893244416,Earth: Senseless nonsense,Pacific Time (US & Canada),""" i'm getting lot hatemail tonight. 😆 gopd...",😆
365,366,No candidate mentioned,0.3940,yes,0.6277,Neutral,0.6277,None of the above,0.3940,NaN,...,NaN,NaN,Speed Dating For A New President? Interesting ...,NaN,2015-08-07 09:52:03 -0700,629696514932084736,"Las Vegas, NV",Pacific Time (US & Canada),speed dating new president? interesting articl...,:-)
496,497,No candidate mentioned,0.4074,yes,0.6383,Positive,0.3404,None of the above,0.4074,NaN,...,NaN,NaN,Kind of glad I didn't get to watch the #GOPDeb...,NaN,2015-08-07 09:50:52 -0700,629696217660784640,seattle,Pacific Time (US & Canada),kind glad didn't get watch gopdebate yesterda...,😊


,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone,processed_text,emoticons
12613,12614,Jeb Bush,1.0,yes,1.0,Negative,1.0,None of the above,1.0,NaN,...,NaN,NaN,😂 RT @RWSurferGirl: Jeb Bush reminds me of el...,NaN,2015-08-06 19:18:41 -0700,629476728470171648,CT,Eastern Time (US & Canada),😂 jeb bush reminds elevator music. hear don'...,"😂 , 🇺🇸"
